# ID2209 Final Project - Challenge - Multiple Auctions

Auction categories are set by entering into auction_categories list

Number of bidders is set by changing numbidders variable



Autogen with ChatGPT configuration

In [1]:
!pip install pyautogen
import autogen
config_list_gpt_35 = [
  {
    "model": "gpt-3.5-turbo",
    "api_version": "2023-03-01-preview",
    "api_key": "<OPENAI_API_KEY>",
  }
]


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.2/295.2 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
tensorf

#Construct Agents

In [2]:
llm_config = {"config_list": config_list_gpt_35, "cache_seed": 14}

In [3]:
bidder_names = [] #list of bidder names
numbidders = 25 #number of bidders who'd like to participate in an auction
auction_categories = ['paintings', 'statues', 'jewellery'] #categories for auctions (also the number of auctions that occur)
numauctions = len(auction_categories)
bidders = [] #list of bidders
auctioneers = [] #list of auctioneers, 1 for each auction category


#creates number of bidders specified by user and gives them instructions
for i in range(numbidders):
  bidder_names.append("bidder" + str(i))
  bidders.append(autogen.AssistantAgent(
    name=bidder_names[i],
    system_message = """Your name is bidder""" + str(i) + """and you are participating in a vickery auction as a bidder.
The auction is conducted by {auction_agent}.
You will first be asked if you are interested in an auction for the categories """ + str(auction_categories) + """. Please pick one randomly when asked.
Just return the category if asked
You should start with a randomly generated bid value in the range of 500-1000 and send it to the auctioneer when requested
Do not include extra chit-chat
""",
    llm_config=llm_config,
    max_consecutive_auto_reply = 1
  ))

#creates auctioneers for each auction category and gives them instructions
for i in range(numauctions):
  auctioneers.append(autogen.AssistantAgent(
    name=auction_categories[i] +  "_AUCTIONEER",
    system_message=""""You are an AI-powered Auctioneer.
You cooridinate a Vickery auction.
The category of your auction is """ + auction_categories[i] + """
You receive bids from the different bidders.
The winner is the agent with the higest bid and the amount they must pay is the second highest bid.
when asked who won the auction return the answer in this format: Winner_name, int(amount).
If there is a problem just return the word error and explain why there was an error.
""",
    llm_config=llm_config,
    max_consecutive_auto_reply = 3
))


#creates a human admin for 'administrative' questions
user_proxy = autogen.UserProxyAgent(
   name="User_proxy",
   system_message="A human admin.",
   human_input_mode="NEVER"
)

#FIPA Propose Interaction Protocol:
User-proxy is initiator

Bidders are participants

The user-proxy sends a proposal for possible auctions

The bidders send a response (either picking the auction or refusing to enter)

http://www.fipa.org/specs/fipa00036/XC00036F.html

In [4]:
categories = [] #list to store the category preference of each potential bidder

#sorted lists of bidders and bidder names for each category
paintings_bidders = []
jewelry_bidders = []
statues_bidders = []
paintings_bidders_names = []
jewelry_bidders_names = []
statues_bidders_names = []

#User proxy asks each potential bidder which category they're interested in
category_message = "The categories for the auction are " + str(auction_categories) + ". Return your name and the category you are interested in?"
for i in range(numbidders):
  bidders[i].clear_history()
  user_proxy.send(
    message = category_message,
    recipient = bidders[i],
    request_reply= True
  )
  categories.append(bidders[i].last_message())

# Iterate through results and split bidders into different lists based on category
i = 0
for result in categories:
    if 'paintings' in result['content']:
        paintings_bidders.append(bidders[i])
        paintings_bidders_names.append(bidder_names[i])
    elif 'jewellery' in result['content']:
        jewelry_bidders.append(bidders[i])
        jewelry_bidders_names.append(bidder_names[i])
    elif 'statues' in result['content']:
        statues_bidders.append(bidders[i])
        statues_bidders_names.append(bidder_names[i])
    i=i+1

#creates combined list of bidders and their names in each auction
auctions = [paintings_bidders, statues_bidders, jewelry_bidders]
auctions_names = [paintings_bidders_names, statues_bidders_names, jewelry_bidders_names]

User_proxy (to bidder0):

The categories for the auction are ['paintings', 'statues', 'jewellery']. Return your name and the category you are interested in?

--------------------------------------------------------------------------------
bidder0 (to User_proxy):

My name is bidder0 and I am interested in the category 'paintings'.

--------------------------------------------------------------------------------
User_proxy (to bidder1):

The categories for the auction are ['paintings', 'statues', 'jewellery']. Return your name and the category you are interested in?

--------------------------------------------------------------------------------
bidder1 (to User_proxy):

My name is bidder1 and I am interested in the category 'paintings'.

--------------------------------------------------------------------------------
User_proxy (to bidder2):

The categories for the auction are ['paintings', 'statues', 'jewellery']. Return your name and the category you are interested in?

------------

#Single Round FIPA English Auction Interaction Protocol
Initiator is the auctioneer

Participants are the bidders interested in corresponding auction

The auctioneer starts the auction and calls for proposals

http://www.fipa.org/specs/fipa00031/XC00031F.html

https://ceur-ws.org/Vol-627/lads_1.pdf

In [5]:
#lists to store bidder and bid for each auction
paintings_bids = []
statues_bids = []
jewelry_bids = []
bids = [paintings_bids,statues_bids,jewelry_bids]

#Each auctioneer tells bidders in their auction that it has begun and to submit their bid
opening_message = "Auction starting. Please send me your bid and your name."
for j in range(numauctions):
  for i in range(len(auctions[j])):
    auctions[j][i].clear_history()
    auctioneers[j].send(
      message = opening_message,
      recipient = auctions[j][i],
      request_reply= True
    )
    result = auctions[j][i].last_message()
    if(result['content'] != opening_message):
      bids[j].append(result)


paintings_AUCTIONEER (to bidder0):

Auction starting. Please send me your bid and your name.

--------------------------------------------------------------------------------
bidder0 (to paintings_AUCTIONEER):

My bid is 720. My name is bidder0.

--------------------------------------------------------------------------------
paintings_AUCTIONEER (to bidder1):

Auction starting. Please send me your bid and your name.

--------------------------------------------------------------------------------
bidder1 (to paintings_AUCTIONEER):

My bid is 750.
My name is bidder1.

--------------------------------------------------------------------------------
paintings_AUCTIONEER (to bidder4):

Auction starting. Please send me your bid and your name.

--------------------------------------------------------------------------------
bidder4 (to paintings_AUCTIONEER):

My bid is 800. My name is bidder4.

--------------------------------------------------------------------------------
paintings_AUCTIO

#Bid summarization
This section is just for clarity

In [6]:
#Asks each auctioneer to summarize their bids (just for clarity)
for j in range(numauctions):
    auctioneers[j].clear_history()
    user_proxy.clear_history()
    instruction_message = "The bids received were "+ str(bids[j]) + "\nPlease summarize the bidders and their bids in a clear list"
    user_proxy.send(

    message = instruction_message,
    recipient = auctioneers[j],
    request_reply = True)

User_proxy (to paintings_AUCTIONEER):

The bids received were [{'content': 'My bid is 720. My name is bidder0.', 'role': 'assistant'}, {'content': 'My bid is 750.\nMy name is bidder1.', 'role': 'assistant'}, {'content': 'My bid is 800. My name is bidder4.', 'role': 'assistant'}, {'content': 'My bid is 750 and my name is bidder5.', 'role': 'assistant'}, {'content': 'My bid is 750 and my name is bidder8.', 'role': 'assistant'}, {'content': 'My bid is 750 and my name is bidder9.', 'role': 'assistant'}, {'content': 'My bid is 750 and my name is bidder11.', 'role': 'assistant'}, {'content': '500, bidder13', 'role': 'assistant'}, {'content': 'My bid is 750. My name is bidder15.', 'role': 'assistant'}, {'content': 'My name is bidder16 and my initial bid is $700.', 'role': 'assistant'}, {'content': 'My bid is 700 and my name is bidder17.', 'role': 'assistant'}, {'content': 'My bid is 750 and my name is bidder18.', 'role': 'assistant'}, {'content': 'My bid is 750 and my name is bidder21.', 'rol

#Single Round FIPA English Auction Interaction Protocol (continued)
Auctioneer accepts the winning proposal and rejects all others

Informs winner of their win and how much they owe

Informs losers the auction is over

In [8]:
#ask each auctioneer to pick their winner, display the results, inform winner of what they owe and inform losers auction is over
for j in range(numauctions):
  auctioneers[j].clear_history()
  user_proxy.clear_history()
  instruction_message = "The bids received were "+ str(bids[j]) + "\nReturn the bidder who placed the highest bid and the value of the second highest bid amount in this format: (Winner_name, int(amount_to_pay)). Provide only the answer"
  user_proxy.send(

    message = instruction_message,
    recipient = auctioneers[j],
    request_reply = True)
  winning_message_info = auctioneers[j].last_message(user_proxy)
  winning_message_info = winning_message_info['content']
  if(winning_message_info != instruction_message):
    value = winning_message_info
  else:
    value = 'error'
  if('error' in value or 'Error' in value):
    print('There was an error, please try again')
  else:
    print(value)
    # Remove parentheses and split the string
    value = value.strip('()')
    bidder, payment = value.split(', ')

    # Strip any extra spaces around the values
    bidder = bidder.strip()
    payment = payment.strip()

    # Extracted bidder and payment
    extracted_bidder = bidder  # Replace this with the extracted bidder variable

    # Notify the winner
    if extracted_bidder in bidder_names:
        winner_index = auctions_names[j].index(extracted_bidder)
        auctioneers[j].send(
          message = "You won the auction for " + auction_categories[j] + ". You must pay $" + payment,
          recipient = auctions[j][winner_index],
          request_reply= True
        )

    # Notify the rest of the bidders
    for index, bidder in enumerate(auctions[j]):
        if index != winner_index:
          auctioneers[j].send(
          message = "The auction for " + auction_categories[j] +" is over. You have not won",
          recipient = auctions[j][index],
          request_reply= False
          )


User_proxy (to paintings_AUCTIONEER):

The bids received were [{'content': 'My bid is 720. My name is bidder0.', 'role': 'assistant'}, {'content': 'My bid is 750.\nMy name is bidder1.', 'role': 'assistant'}, {'content': 'My bid is 800. My name is bidder4.', 'role': 'assistant'}, {'content': 'My bid is 750 and my name is bidder5.', 'role': 'assistant'}, {'content': 'My bid is 750 and my name is bidder8.', 'role': 'assistant'}, {'content': 'My bid is 750 and my name is bidder9.', 'role': 'assistant'}, {'content': 'My bid is 750 and my name is bidder11.', 'role': 'assistant'}, {'content': '500, bidder13', 'role': 'assistant'}, {'content': 'My bid is 750. My name is bidder15.', 'role': 'assistant'}, {'content': 'My name is bidder16 and my initial bid is $700.', 'role': 'assistant'}, {'content': 'My bid is 700 and my name is bidder17.', 'role': 'assistant'}, {'content': 'My bid is 750 and my name is bidder18.', 'role': 'assistant'}, {'content': 'My bid is 750 and my name is bidder21.', 'rol